# *Balances de materia*
## Resolución de un balance con reacción y recirculación

### El diagrama de bloques del proceso es el siguiente:


<img src="./Imagenes/Flujo_Recirculacion.jpg" height="400" width="400"/>



In [1]:
%matplotlib inline
import numpy as np
from scipy import optimize
from scipy.integrate import ode
import matplotlib.pyplot as mpl
import matplotlib.patches as patches
import random
import math
import pandas as pd
import IPython.core.display as di
from ipywidgets import interact
import notebook
from IPython.display import clear_output, display, HTML, Image,Math, Latex
from IPython.external import mathjax

#Generador de reaccion aA+bB->cD+dD

def generador_estequiometria():  # generación de parámetros del problema

    coeficientes={'a': None, 'b': None,'c': None,'d': None,}
    Pesos_moleculares={'a': 0, 'b': 0,'c': 0,'d': 0,}

    while (Pesos_moleculares['d']<=10 or (coeficientes['a']==0 or coeficientes['b']==0)):

        for i in coeficientes.keys():
            coeficientes[i]= random.choice([0,1,2,3])
            Pesos_moleculares[i]= round(random.uniform(10.,80.),1)

        if coeficientes['d']!=0:

            Pesos_moleculares['d']=round((Pesos_moleculares['a']*coeficientes['a']+Pesos_moleculares['b']*coeficientes['b']-\
                                Pesos_moleculares['c']*coeficientes['c'])/coeficientes['d'],1)
        elif  (coeficientes['d']==0 and (coeficientes['a']!=0 and coeficientes['b']!=0)):
            Pesos_moleculares['d']=1
            break

    return coeficientes,Pesos_moleculares

def Reactivo_limitante(C2, W2_a,W2_b,coeficientes,Pesos_moleculares):

    a=coeficientes['a']; b=coeficientes['b']
    PM_a=Pesos_moleculares['a']; PM_b=Pesos_moleculares['b']

    if (C2*W2_a)/(PM_a)<=(C2*W2_b*a/(b*PM_b)):
        Limitante='a'
    else:
        Limitante = 'b'

    return Limitante


def generador_parametros():  # generación de parámetros del problema
    C2,W2_d ,C5=0,0,0

    while C2 <= 1.5 * C5:
        C2 = random.randint(10., 200.)  # Caudal másico kg/h
        C5 = random.randint(10., 200.)  # Caudal másico kg/h

    while W2_d <= 0:  # fracciones másicas de la corriente 2
        W2_a = random.uniform(0.2, 0.6)
        W2_b = random.uniform(0.2, 0.6)
        W2_c = random.uniform(0, 0.15)
        W2_d = 1-(W2_a + W2_b + W2_c)

    return C2, C5, W2_a, W2_b, W2_c,W2_d


def generador_conversion(Limitante, W2_a,W2_b,coeficientes,Pesos_moleculares):

    a=coeficientes['a'];b=coeficientes['b']
    PM_a=Pesos_moleculares['a'];PM_b=Pesos_moleculares['b']

    while True:
        conversion = random.uniform(0.5, 1.0)

        if Limitante=='a':
            return conversion
        else:
            conversion_a=(a/b)*(W2_b*PM_a*conversion)/(W2_a*PM_b)
            #print (conversion)
            if conversion_a>=0.01 and conversion_a<=1:
                return  conversion_a



def Sistema_Ecuaciones(incognitas,variables_conocidas,coeficientes,Pesos_moleculares,conversion_A):
    #Variables conocidas y desconocidas dependen del caso pero se resuelve siempre para las mismas conocidas
    # y generadas.
    #Variables del sistema: C1,C2,C3,C4,C5
    # W1_a, W1_b, W1_c, W1_d
    # W2_a, W2_b, W2_c, W2_d
    # W3_a, W3_b, W3_c, W3_d
    # W4_a, W4_b, W4_c, W4_d
    # W5_a, W5_b, W5_c, W5_d

    # W3_a=W4_a=W5_a Por lo que se toma en todas las ecuaciones w3_a
    # W3_b=W4_b=W5_b Por lo que se toma en todas las ecuaciones w3_b
    # W3_c=W4_c=W5_c Por lo que se toma en todas las ecuaciones w3_c
    # W3_d=W4_d=W5_d Por lo que se toma en todas las ecuaciones w3_d

    # 14 variables de entrada, 5 de ellas definidas, y 9 ecuaciones y 9 incognitas
    # Siempre se general, además de la estequiometría y la conversión, el caudal y 3 composiciones de C2 y C5
    # Una vez resuelto, según el caso, se muestran al estudiante unos valores de partida u otros

    X=conversion_A

    a=coeficientes['a'];b=coeficientes['b'];c=coeficientes['c']
    PM_a=Pesos_moleculares['a'];PM_b=Pesos_moleculares['b'];PM_c=Pesos_moleculares['c']

    C1,C3,C4,W1_a, W1_b, W1_c,W3_a, W3_b, W3_c =incognitas
    C2, C5, W2_a, W2_b, W2_c=variables_conocidas

    #Ecuaciones en el mezclador
    values=[C1*W1_a+C5*W3_a-C2*W2_a] #Ecuación 1
    values.append(C1*W1_b+C5*W3_b-C2*W2_b)#Ecuación 2
    values.append(C1*W1_c+C5*W3_c-C2*W2_c) #Ecuación 3
    values.append(C1+C5-C2)#Ecuación 4 - balance Global

    #Ecuaciones en el divisor
    values.append(C3-C4-C5) #Ecuación 5 Balance global


    #Ecuaciones en el reactor
    values.append(C3*W3_a/PM_a+C2*W2_a*X/PM_a-C2*W2_a/PM_a) #Ecuación 6
    values.append(C3*W3_b/PM_b+((b/a)*C2*W2_a*X)/PM_a-C2*W2_b/PM_b) #Ecuación 7
    values.append(C3*W3_c/PM_c-((c/a)*C2*W2_a*X)/PM_a-C2*W2_c/PM_c) #Ecuación 8
    values.append(C3-C2) #Ecuación 9 Balance global

    return values

def generador_problema():
    
    comprobador=False
    
    while comprobador==False:
        C2, C5, W2_a, W2_b, W2_c,W2_d =generador_parametros()
        
        coeficientes,Pesos_moleculares=generador_estequiometria()
        
        Limitante=Reactivo_limitante(C2, W2_a,W2_b,coeficientes,Pesos_moleculares)
        
        conversion=generador_conversion(Limitante, W2_a,W2_b,coeficientes,Pesos_moleculares)
        
        variables_conocidas = [C2, C5, W2_a, W2_b, W2_c]
        
        Resultado = optimize.fsolve(Sistema_Ecuaciones, [100, 100, 100, 0.3, 0.3, 0.1,0.3, 0.3, 0.1],\
                                  args=(variables_conocidas,coeficientes,Pesos_moleculares,conversion),\
                                 xtol=1e-06, maxfev=500)
        
        C1,C3,C4,W1_a, W1_b, W1_c,W3_a, W3_b, W3_c  = Resultado
        
        W1_d = 1-(W1_a + W1_b + W1_c)
        W3_d = 1-(W3_a + W3_b + W3_c)
        R=C5/C3
        corrientes=[C1,C2, C3,C4,C5]
        fracciones=[ W1_a, W1_b, W1_c,W1_d , W2_a, W2_b, W2_c,W2_d ,W3_a, W3_b, W3_c,  W3_d,R ]
        corrientes=[int(elem) for elem in corrientes]
        fracciones=[round(float(elem), 3) for elem in fracciones]
        comprobador=all(i >= 0 for i in fracciones)
    
    return fracciones,corrientes, coeficientes,Pesos_moleculares,round(conversion,3)

fracciones,corrientes,coeficientes,Pesos_moleculares,conversion =generador_problema()

W1_a, W1_b, W1_c,W1_d , W2_a, W2_b, W2_c,W2_d ,W3_a, W3_b, W3_c,  W3_d,R=fracciones

C1,C2, C3,C4,C5=corrientes

#Definir casos: generar variables y sus valores para mostrarlos

caso=random.randint(1,4)

if caso==1:
    variablesConocidasNombres=['C2 (kg/h)','C5 (kg/h)',' W2_a ',' W2_b ',' W2_c ',' XA ']
    ValoresMostrados=[C2, C5, W2_a, W2_b, W2_c,conversion]

    Pesos_Moleculares_Nombres=['PM_a (g/mol)','PM_b (g/mol)','PM_c (g/mol)','PM_d (g/mol)']
    ValoresMostradosPM=[Pesos_moleculares['a'],Pesos_moleculares['b'],Pesos_moleculares['c'],Pesos_moleculares['d']]

    coeficientesNombres=['a','b','c','d']
    ValoresCoeficientes=[coeficientes['a'],coeficientes['b'],coeficientes['c'],coeficientes['d']]

    ResultadosNombres=['C1 (kg/h)','C3 (kg/h)','C4 (kg/h)',' W1_a ',' W1_b ',' W1_c ',' W1_d ',\
                       ' W3_a ',' W3_b ',' W3_c ',' W3_d' ]
    ValoresResultados=[C1, C3,C4,W1_a, W1_b, W1_c,W1_d ,W3_a, W3_b, W3_c,  W3_d]

elif caso==2:
    variablesConocidasNombres=['C1 (kg/h)',' W1_a ',' W1_b ',' W1_c ',' R ',' XA ']
    ValoresMostrados=[C1,W1_a, W1_b, W1_c,R,conversion]

    Pesos_Moleculares_Nombres=['PM_a (g/mol)','PM_b (g/mol)','PM_c (g/mol)','PM_d (g/mol)']
    ValoresMostradosPM=[Pesos_moleculares['a'],Pesos_moleculares['b'],Pesos_moleculares['c'],Pesos_moleculares['d']]

    coeficientesNombres=['a','b','c','d']
    ValoresCoeficientes=[coeficientes['a'],coeficientes['b'],coeficientes['c'],coeficientes['d']]

    ResultadosNombres=['C2 (kg/h)','C3 (kg/h)','C4 (kg/h)','C5 (kg/h)',' W2_a ',' W2_b ',' W2_c ',' W2_d ',\
                       ' W3_a ',' W3_b ',' W3_c ',' W3_d' ]
    ValoresResultados=[C2, C3,C4,C5, W2_a, W2_b, W2_c,W2_d ,W3_a, W3_b, W3_c,  W3_d]
    
elif caso==3:
    variablesConocidasNombres=['C3 (kg/h)',' W3_a ',' W3_b ',' W3_c ',' R ',' XA ']
    ValoresMostrados=[C3,W3_a, W3_b, W3_c,R,conversion]

    Pesos_Moleculares_Nombres=['PM_a (g/mol)','PM_b (g/mol)','PM_c (g/mol)','PM_d (g/mol)']
    ValoresMostradosPM=[Pesos_moleculares['a'],Pesos_moleculares['b'],Pesos_moleculares['c'],Pesos_moleculares['d']]

    coeficientesNombres=['a','b','c','d']
    ValoresCoeficientes=[coeficientes['a'],coeficientes['b'],coeficientes['c'],coeficientes['d']]

    ResultadosNombres=['C1 (kg/h)','C2 (kg/h)','C4 (kg/h)','C5 (kg/h)',' W1_a ',' W1_b ',' W1_c ',' W1_d ',\
                       ' W2_a ',' W2_b ',' W2_c ',' W2_d ']
    ValoresResultados=[C1, C2,C4,C5, W1_a,W1_b, W1_c,W1_d , W2_a, W2_b, W2_c,W2_d ]
 
elif caso==4:
    variablesConocidasNombres=['C5 (kg/h)',' W5_a ',' W5_b ',' W5_c ',' R ',' XA ']
    ValoresMostrados=[C5,W3_a, W3_b, W3_c,R,conversion]

    Pesos_Moleculares_Nombres=['PM_a (g/mol)','PM_b (g/mol)','PM_c (g/mol)','PM_d (g/mol)']
    ValoresMostradosPM=[Pesos_moleculares['a'],Pesos_moleculares['b'],Pesos_moleculares['c'],Pesos_moleculares['d']]

    coeficientesNombres=['a','b','c','d']
    ValoresCoeficientes=[coeficientes['a'],coeficientes['b'],coeficientes['c'],coeficientes['d']]

    ResultadosNombres=['C1 (kg/h)','C2 (kg/h)','C3 (kg/h)','C4 (kg/h)',' W1_a ',' W1_b ',' W1_c ',' W1_d ',\
                       ' W2_a ',' W2_b ',' W2_c ',' W2_d ']
    ValoresResultados=[C1, C2,C3,C4, W1_a,W1_b, W1_c,W1_d , W2_a, W2_b, W2_c,W2_d ]
 

    
display(HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Mostrar código')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Esconder código')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Mostrar código"></form>'''))



/Users/juancarlos/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/juancarlos/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
display(HTML('<h1 style="color:#000000"><strong>La reacción que tiene lugar en el reactor es:</strong></h1>'))

display(Math(r'a\cdot A+b\cdot B \rightarrow c\cdot C+d\cdot D'))

display(HTML('<h1 style="color:#000000"><strong>Los coeficiences estequiométricos de la reacción son:</strong></h1>'))

data2 = dict(zip(coeficientesNombres, ValoresCoeficientes))

values2 = pd.DataFrame(data2,index=['Coef. Estequiométricos'], columns=coeficientesNombres)

display(values2)


display(HTML('<p style="color:#000000"><strong>Nomenclatura</strong>: a, b, c y d son los coeficientes\
 estequiométricos de los compuestos A, B, C y D, respectivamente.</p>'))

<IPython.core.display.Math object>

a  b  c  d
Coef. Estequiométricos  1  1  0  0

In [3]:
display(HTML('<h1 style="color:#000000"><strong>Los datos del problema son:</strong></h1>'))

data = dict(zip(variablesConocidasNombres, ValoresMostrados))

values = pd.DataFrame(data,index=['Datos'], columns=variablesConocidasNombres)

display(values)


data1 = dict(zip(Pesos_Moleculares_Nombres, ValoresMostradosPM))

values1 = pd.DataFrame(data1,index=['Pesos Moleculares'], columns=Pesos_Moleculares_Nombres)

display(values1)

display(HTML('<p style="color:#000000"><strong>Nomenclatura</strong>: el prefijo C corresponde a una corriente,\
W#_letra denota una fracción másica en la corriente # para el compuesto "letra", \
R es la razón de recirculación, XA es la conversión para el compuesto A, \
PM_letra es el peso molecular del compuesto )</p>'))


C2 (kg/h)  C5 (kg/h)   W2_a    W2_b    W2_c    XA 
Datos        114         25   0.235   0.525   0.079  0.86

PM_a (g/mol)  PM_b (g/mol)  PM_c (g/mol)  PM_d (g/mol)
Pesos Moleculares          54.4          68.6          17.6             1

In [4]:
display(HTML('<h1 style="color:#000000"><strong>Los resultados son:</strong></h1>'))

data3 = dict(zip(ResultadosNombres, ValoresResultados))

values3 = pd.DataFrame(data3,index=['Resultados'], columns=ResultadosNombres)

display(values3)

display(HTML('<p style="color:#000000">Las fracciones másicas en las corrientes 3, 4 y 5 son \
iguales, dado que se trata de un mezclador.</p>'))



C1 (kg/h)  C3 (kg/h)  C4 (kg/h)   W1_a    W1_b    W1_c    W1_d   \
Resultados         89        114         89   0.291   0.596   0.079   0.033   

             W3_a    W3_b    W3_c    W3_d  
Resultados   0.033    0.27   0.079  0.618

In [5]:
display(HTML('''

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Programado con Jupyter Notebook en Python 3.6. </footer>'''))